Red Wine Classification Proposal

By DSCI 100, Group 39

This project aims to classify bottles of wine based on their chemical properties and identify the accuracy with which the predictions are made. 

In [17]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [1]:
#read the dataset in Jupyter
red_wine <- read.csv2("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv")
red_wine

fixed.acidity volatile.acidity citric.acid residual.sugar chlorides
1    7.4           0.7              0           1.9            0.076    
2    7.8           0.88             0           2.6            0.098    
3    7.8           0.76             0.04        2.3            0.092    
4    11.2          0.28             0.56        1.9            0.075    
5    7.4           0.7              0           1.9            0.076    
6    7.4           0.66             0           1.8            0.075    
7    7.9           0.6              0.06        1.6            0.069    
8    7.3           0.65             0           1.2            0.065    
9    7.8           0.58             0.02        2              0.073    
10   7.5           0.5              0.36        6.1            0.071    
11   6.7           0.58             0.08        1.8            0.097    
12   7.5           0.5              0.36        6.1            0.071    
13   5.6           0.615            0           1.6            0.089    
14   7.8           0.61             0.29        1.6            0.114    
15   8.9           0.62             0.18        3.8            0.176    
16   8.9           0.62             0.19        3.9            0.17     
17   8.5           0.28             0.56        1.8            0.092    
18   8.1           0.56             0.28        1.7            0.368    
19   7.4           0.59             0.08        4.4            0.086    
20   7.9           0.32             0.51        1.8            0.341    
21   8.9           0.22             0.48        1.8            0.077    
22   7.6           0.39             0.31        2.3            0.082    
23   7.9           0.43             0.21        1.6            0.106    
24   8.5           0.49             0.11        2.3            0.084    
25   6.9           0.4              0.14        2.4            0.085    
26   6.3           0.39             0.16        1.4            0.08     
27   7.6           0.41             0.24        1.8            0.08     
28   7.9           0.43             0.21        1.6            0.106    
29   7.1           0.71             0           1.9            0.08     
30   7.8           0.645            0           2              0.082    
⋮    ⋮             ⋮                ⋮           ⋮              ⋮        
1570 6.2           0.51             0.14        1.9            0.056    
1571 6.4           0.36             0.53        2.2            0.23     
1572 6.4           0.38             0.14        2.2            0.038    
1573 7.3           0.69             0.32        2.2            0.069    
1574 6             0.58             0.2         2.4            0.075    
1575 5.6           0.31             0.78        13.9           0.074    
1576 7.5           0.52             0.4         2.2            0.06     
1577 8             0.3              0.63        1.6            0.081    
1578 6.2           0.7              0.15        5.1            0.076    
1579 6.8           0.67             0.15        1.8            0.118    
1580 6.2           0.56             0.09        1.7            0.053    
1581 7.4           0.35             0.33        2.4            0.068    
1582 6.2           0.56             0.09        1.7            0.053    
1583 6.1           0.715            0.1         2.6            0.053    
1584 6.2           0.46             0.29        2.1            0.074    
1585 6.7           0.32             0.44        2.4            0.061    
1586 7.2           0.39             0.44        2.6            0.066    
1587 7.5           0.31             0.41        2.4            0.065    
1588 5.8           0.61             0.11        1.8            0.066    
1589 7.2           0.66             0.33        2.5            0.068    
1590 6.6           0.725            0.2         7.8            0.073    
1591 6.3           0.55             0.15        1.8            0.077    
1592 5.4           0.74             0.09        1.7            

In [19]:
#make quality (originally ranked from 1-10) a categorical variable 
red_wine <- red_wine %>%
    mutate(quality = case_when(quality == "10" ~ "Excellent",
                              quality == "9" ~ "Excellent",
                              quality == "8" ~ "Good",
                              quality == "7" ~ "Good",
                              quality == "6" ~ "Satisfactory",
                              quality == "5" ~ "Satisfactory",
                              quality == "4" ~ "Bad",
                              quality == "3" ~ "Bad",
                              quality == "2" ~ "Poor",
                              quality == "1" ~ "Poor"))

In [20]:
#we will be treating quality as a categorical variable, so we converted into a factor
#we converted the rest of the variables into numerical variables
red_wine <- red_wine %>% 
    mutate(fixed.acidity = as.numeric(fixed.acidity)) %>% 
    mutate(volatile.acidity = as.numeric(volatile.acidity)) %>% 
    mutate(citric.acid = as.numeric(citric.acid)) %>% 
    mutate(residual.sugar = as.numeric(residual.sugar)) %>% 
    mutate(chlorides = as.numeric(chlorides)) %>% 
    mutate(free.sulfur.dioxide = as.numeric(free.sulfur.dioxide)) %>% 
    mutate(total.sulfur.dioxide = as.numeric(total.sulfur.dioxide)) %>% 
    mutate(density = as.numeric(density)) %>% 
    mutate(pH = as.numeric(pH)) %>% 
    mutate(sulphates = as.numeric(sulphates)) %>% 
    mutate(alcohol = as.numeric(alcohol)) %>% 
    mutate(quality = as.factor(quality))

In [21]:
#create a filtered dataset by quality for each classification
#note that red_wine has no inputs for "Excellent" and "Poor"
good_class <- red_wine %>% 
    filter(quality == "Good")
good_class

satisfactory_class <- red_wine %>% 
    filter(quality == "Satisfactory")
satisfactory_class

bad_class <- red_wine %>% 
    filter(quality == "Bad")
bad_class

fixed.acidity volatile.acidity citric.acid residual.sugar chlorides
1   7.3           0.65             0.00        1.2            0.065    
2   7.8           0.58             0.02        2.0            0.073    
3   8.5           0.28             0.56        1.8            0.092    
⋮   ⋮             ⋮                ⋮           ⋮              ⋮        
215 7.4           0.36             0.30        1.8            0.074    
216 7.0           0.56             0.17        1.7            0.065    
217 6.7           0.32             0.44        2.4            0.061    
    free.sulfur.dioxide total.sulfur.dioxide density pH   sulphates alcohol
1   15                   21                  0.9946  3.39 0.47      10.0   
2    9                   18                  0.9968  3.36 0.57       9.5   
3   35                  103                  0.9969  3.30 0.75      10.5   
⋮   ⋮                   ⋮                    ⋮       ⋮    ⋮         ⋮      
215 17                  24                   0.99419 3.24 0.70      11.40  
216 15                  24                   0.99514 3.44 0.68      10.55  
217 24                  34                   0.99484 3.29 0.80      11.60  
    quality
1   Good   
2   Good   
3   Good   
⋮   ⋮      
215 Good   
216 Good   
217 Good

fixed.acidity volatile.acidity citric.acid residual.sugar chlorides
1    7.4           0.70             0.00        1.9            0.076    
2    7.8           0.88             0.00        2.6            0.098    
3    7.8           0.76             0.04        2.3            0.092    
⋮    ⋮             ⋮                ⋮           ⋮              ⋮        
1317 6.3           0.510            0.13        2.3            0.076    
1318 5.9           0.645            0.12        2.0            0.075    
1319 6.0           0.310            0.47        3.6            0.067    
     free.sulfur.dioxide total.sulfur.dioxide density pH   sulphates alcohol
1    11                  34                   0.9978  3.51 0.56      9.4    
2    25                  67                   0.9968  3.20 0.68      9.8    
3    15                  54                   0.9970  3.26 0.65      9.8    
⋮    ⋮                   ⋮                    ⋮       ⋮    ⋮         ⋮      
1317 29                  40                   0.99574 3.42 0.75      11.0   
1318 32                  44                   0.99547 3.57 0.71      10.2   
1319 18                  42                   0.99549 3.39 0.66      11.0   
     quality     
1    Satisfactory
2    Satisfactory
3    Satisfactory
⋮    ⋮           
1317 Satisfactory
1318 Satisfactory
1319 Satisfactory

fixed.acidity volatile.acidity citric.acid residual.sugar chlorides
1  7.4           0.59             0.08        4.4            0.086    
2  5.7           1.13             0.09        1.5            0.172    
3  8.8           0.61             0.30        2.8            0.088    
⋮  ⋮             ⋮                ⋮           ⋮              ⋮        
61 6.8           0.91             0.06        2.0            0.060    
62 6.7           0.76             0.02        1.8            0.078    
63 6.9           0.48             0.20        1.9            0.082    
   free.sulfur.dioxide total.sulfur.dioxide density pH   sulphates alcohol
1   6                  29                   0.9974  3.38 0.50      9.0    
2   7                  19                   0.9940  3.50 0.48      9.8    
3  17                  46                   0.9976  3.26 0.51      9.3    
⋮  ⋮                   ⋮                    ⋮       ⋮    ⋮         ⋮      
61 4                   11                   0.99592 3.53 0.64      10.90  
62 6                   12                   0.99600 3.55 0.63       9.95  
63 9                   23                   0.99585 3.39 0.43       9.05  
   quality
1  Bad    
2  Bad    
3  Bad    
⋮  ⋮      
61 Bad    
62 Bad    
63 Bad

In [42]:
options(repr.plot.width = 12, repr.plot.height = 8)
initial_plot <- red_wine %>% 
    ggplot(aes(x = alcohol, y = pH, color = quality)) +
    geom_point() +
    labs(x = "Red Wine Alcohol Level", y = "Red Wine pH Level", color = "Red Wine Quality") +
    ggtitle("Relationship Between Red Wine Alcohol and pH Levels") +
    theme(text = element_text(size = 20))
initial_plot

## Method
Regarding variables, we will use ? and quality as the prediction result.
However, we want to take quality as factor, so we created 5 standard regarding different quality.
We made 'Excellent' , 'Good', 'Satisfactory', 'Bad' and 'Poor'regarding different quality.

Regarding how we will conduct data analysis. We will split our data to training data and test data, for training data we will do cross validation and use the nearest neighbour model to tune the K. Once we get our best model, we will predict the result and compare the result to our test data to see the accuracy of this model and also check the confusion matrix.

We will visualize the result as scatter plot graph accuracy vs. the number of neighbours since in this case we can see how k affects the accuracy of prediction.

(Discuss about variables?)


## Expected Outcomes and Significance:

We expect to create a model that classifies a bottle of wine based on its chemical properties and also to identify the accuracy with which it makes these predictions. Our result can have significance for various shareholders such as direct consumers, retailers as well as wine critics who are involved in quality predictions.
These findings would impact the wine industry's production methods and quality assurance methods. If a standardized model is created, wine producers can rely on that instead of a sole wine taster. Manufacturers can alter the chemical constituents of their wines and ensure that their values lie within a very excellent range (as predicted by our model). Consumers’ lives would be impacted in the sense that they would know which red-wine labels to look out for even without trying them out. If the model is accurate the importance of wine critics would reduce as their role would only be limited to providing observations to improve our model. 

The resulting questions from our analysis could be, would the model be able to derive results of the same level of accuracy and applicability for other types of wines? Is the method limited to certain types of sensory and subjective ranking procedures, and what would we define these limitations by? Can the model fully automate the quality testing process for red wine production?
